<a href="https://colab.research.google.com/github/laggykiller/sticker-convert/blob/master/sticker_convert_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Tip: Press the play button on the left to run code { display-mode: "form" }

In [ ]:
#@title Tip: On Google Colab, stop the session after use by going to 'Runtime -> Manage sessions -> Terminate' { display-mode: "form" }

In [ ]:
#@title 1. Getting dependencies { display-mode: "form" }
# Get packages
!apt install ffmpeg optipng pngquant zip gcc make cmake libpng-dev libboost-program-options-dev libboost-regex-dev libboost-system-dev libboost-filesystem-dev build-essential curl unzip pkg-config python3-tk cargo python3-opencv

# Download ImageMagick AppImage
# Version on official repo throws: python3: ../../magick/quantum.c:216: DestroyQuantumInfo: Assertion `quantum_info != (QuantumInfo *) NULL' failed
!curl -O -L https://imagemagick.org/archive/binaries/magick
!chmod +x /content/magick
!mv /content/magick /usr/local/bin

# Download apngasm
!curl -O -L https://launchpad.net/~zero-tsuki/+archive/ubuntu/ppa/+build/20404803/+files/libapngasm_3.1.9~134~ubuntu18.04.1_amd64.deb
!curl -O -L https://launchpad.net/~zero-tsuki/+archive/ubuntu/ppa/+build/20404803/+files/apngasm_3.1.9~134~ubuntu18.04.1_amd64.deb

!dpkg -i libapngasm_3.1.9~134~ubuntu18.04.1_amd64.deb
!dpkg -i apngasm_3.1.9~134~ubuntu18.04.1_amd64.deb

# Compile pngnq-s9
!mkdir /content/pngnq-s9
%cd /content/pngnq-s9
!curl -O -L https://github.com/ImageProcessing-ElectronicPublications/pngnq-s9/archive/refs/tags/2.0.2.tar.gz
!tar xvzf 2.0.2.tar.gz
%cd /content/pngnq-s9/pngnq-s9-2.0.2
# Fix bug that causes compilation to fail
!sed -i '1i#include <string.h>' src/rwpng.c
!./configure
!make
!make install
%cd /content
!rm -rf /content/pngnq-s9

# Compile apngdis
!mkdir /content/apngdis
%cd /content/apngdis
!curl -O -L https://sourceforge.net/projects/apngdis/files/2.9/apngdis-2.9-src.zip
!unzip apngdis-2.9-src.zip
!make
!cp /content/apngdis/apngdis /usr/local/bin
%cd /content
!rm -rf /content/apngdis

# Compile pngquant as current version does not have '--strip' option
%cd /content
!git clone --recursive --branch 2.17.0 https://github.com/kornelski/pngquant.git
%cd /content/pngquant
!cargo build --release
!cp /content/pngquant/target/release/pngquant /usr/local/bin
%cd /content
!rm -rf /content/pngquant

!git clone https://github.com/laggykiller/sticker-convert
%cd /content/sticker-convert
!pip3 install -r requirements.txt

import subprocess
import sys

In [ ]:
#@title 2. Mounting Google Drive { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/stickers_input
!mkdir /content/drive/MyDrive/stickers_output

In [ ]:
#@title 3. Place your files in "stickers_input" folder on your Google Drive { display-mode: "form" }

In [ ]:
#@title 4. Fill in your options and run { display-mode: "form" }
%cd /content/sticker-convert/sticker_convert

#@markdown # Input
#@markdown ### Input source
input_option = 'Download from Telegram' #@param ['Download from Signal', 'Download from Telegram', 'Download from Line', 'Download from Kakao', 'From Google Drive directory']
#@markdown ### Input URL (If applicable)
input_url = '' #@param {type:"string"}
#@markdown ### Input directory
input_dir = '/content/drive/MyDrive/stickers_input' #@param {type:"string"}
#@markdown ---

#@markdown # Compression options
#@markdown ## Preset
compression_preset = 'signal' #@param ['signal', 'telegram', 'whatsapp', 'custom']
#@markdown ## No compression
no_compression = False #@param {type:"boolean"}
#@markdown ## *Use below for overriding preset*
#@markdown ### Output FPS (Minimum)
fps_min = -1 #@param {type:"integer"}
#@markdown ### Output FPS (Maximum)
fps_max = -1 #@param {type:"integer"}
#@markdown ### Output resolution (Minimum)
res_min = -1 #@param {type:"integer"}
#@markdown ### Output resolution (Maximum)
res_max = -1 #@param {type:"integer"}
#@markdown ### Output quality (Minimum)
quality_min = -1 #@param {type:"integer"}
#@markdown ### Output quality (Maximum)
quality_max = -1 #@param {type:"integer"}
#@markdown ### Colors [apng only] (Minimum)
color_min = -1 #@param {type:"integer"}
#@markdown ### Colors [apng only] (Maximum)
color_max = -1 #@param {type:"integer"}
#@markdown ### Maximum file size (Static)
img_size_max = -1 #@param {type:"integer"}
#@markdown ### Maximum file size (Animated)
vid_size_max = -1 #@param {type:"integer"}
#@markdown ### File format (Static)
img_size_max = -1 #@param {type:"integer"}
#@markdown ### File format (Animated)
vid_size_max = -1 #@param {type:"integer"}
#@markdown ### Default emoji
default_emoji = '' #@param {type:"string"}
#@markdown ### Number of steps (Higher = Slower but closer to target size)
steps = -1 #@param {type:"integer"}
#@markdown ### Number of processses (Higher = Compress faster but consume more resources)
processes = -1 #@param {type:"integer"}
#@markdown ---

#@markdown # Output
#@markdown ### Output options
output_option = 'Upload to Signal' #@param ['Upload to Telegram', 'Upload to Signal', 'Compress to .wastickers (WhatsApp)', 'Save to Google Drive directory only']
#@markdown ### Output directory
output_dir = '/content/drive/MyDrive/stickers_output' #@param {type:"string"}
#@markdown ### Title
title = '' #@param {type:"string"}
#@markdown ### Author
author = '' #@param {type:"string"}
#@markdown ---

#@markdown # Credentials
#@markdown ### Signal uuid
signal_uuid = '' #@param {type:"string"}
#@markdown ### Signal password
signal_password = '' #@param {type:"string"}
#@markdown ### Telegram token
telegram_token = '' #@param {type:"string"}
#@markdown ### Telegram user_id
telegram_userid = '' #@param {type:"string"}

params = []

if input_dir != '':
  params.append('--input-dir')
  params.append(input_dir)

if input_option == 'Download from Signal':
  params.append('--download-signal')
  params.append(input_url)
elif input_option == 'Download from Telegram':
  params.append('--download-telegram')
  params.append(input_url)
  if telegram_token == '':
    print('ERROR: Telegram token required for uploading and downloading telegram stickers')
    exit()
elif input_option == 'Download from Line':
  params.append('--download-line')
  params.append(input_url)
elif input_option == 'Download from Kakao':
  params.append('--download-kakao')
  params.append(input_url)

if output_dir != '':
  params.append('--output-dir')
  params.append(output_dir)

if output_option == 'Upload to Telegram':
  params.append('--export-telegram')
  if telegram_token == '':
    print('ERROR: Telegram token required for uploading and downloading telegram stickers')
    exit()
  if telegram_userid == '':
    print('ERROR: Telegram user_id required for uploading telegram stickers')
    exit()
elif output_option == 'Upload to Signal':
  params.append('--export-signal')
  if signal_uuid == '':
    print('ERROR: Signal uuid required for uploading signal stickers')
    exit()
  if signal_password == '':
    print('ERROR: Signal password required for uploading signal stickers')
    exit()
elif output_option == 'Compress to .wastickers (WhatsApp)':
  params.append('--export-wastickers')

if compression_preset != '':
  params.append('--preset')
  params.append(compression_preset)

if no_compression:
  params.append('--no-compress')

if fps_min != -1:
  params.append('--fps-min')
  params.append(str(fps_min))
if fps_max != -1:
  params.append('--fps-max')
  params.append(str(fps_max))
if res_min != -1:
  params.append('--res-min')
  params.append(str(res_min))
if res_max != -1:
  params.append('--res_max')
  params.append(str(res_max))
if quality_min != -1:
  params.append('--quality-min')
  params.append(str(quality_min))
if quality_max != -1:
  params.append('--quality-max')
  params.append(str(quality_max))
if color_min != -1:
  params.append('--color-min')
  params.append(str(color_min))
if color_max != -1:
  params.append('--color-max')
  params.append(str(color_max))
if img_size_max != -1:
  params.append('--img-size-max')
  params.append(str(img_size_max))
if vid_size_max != -1:
  params.append('--vid-size-max')
  params.append(str(vid_size_max))
if default_emoji != '':
  params.append('--default-emoji')
  params.append(str(default_emoji))
if steps != -1:
  params.append('--steps')
  params.append(str(steps))
if processes != -1:
  params.append('--processes')
  params.append(str(processes))
if title != '':
  params.append('--title')
  params.append(str(title))
if title != '':
  params.append('--author')
  params.append(str(author))
if signal_uuid != '':
  params.append('--signal-uuid')
  params.append(str(signal_uuid))
if signal_password != '':
  params.append('--signal-password')
  params.append(str(signal_password))
if telegram_token != '':
  params.append('--telegram-token')
  params.append(str(telegram_token))
if telegram_userid != '':
  params.append('--telegram-userid')
  params.append(str(telegram_userid))

cmds = ['python3', '/content/sticker-convert/sticker_convert/main.py']

cmds += params

print(cmds)

process = subprocess.Popen(cmds, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in process.stdout:
    sys.stdout.write(line)

In [ ]:
#@title 5. Results will be available in "stickers_output" folder on your Google Drive { display-mode: "form" }

In [ ]:
#@title (Run this to get help messages) { display-mode: "form" }
%cd /content/sticker-convert/sticker_convert
!python3 /content/sticker-convert/sticker_convert/main.py --help